In [ ]:
import ee
import time
import sys
import os
import geopandas as gpd
import urllib.request

## Download SRTM

In [ ]:
# =========================================================================
# 0. DEFINIÇÕES DE CAMINHO LOCAL E CONFIGURAÇÃO
# =========================================================================

# Variáveis Locais
ROI_LOCAL_PATH = r'D:\POS\!FW2_HAND_osmnx_4s_op3n\3_Versao_unificada\bases\inundacao_segmentada_por_solo.geojson' # ⬅️ CAMINHO DIRETO
OUTUPUT_FOLDER_MDE = r'srtm' # ⬅️ PASTA DE SAÍDA MANTIDA
EXPORT_DESCRIPTION = 'SRTM_Brumadinho_Raster_30m' # <-- MUDANÇA

# Variáveis do Google Earth Engine
SRTM_ID = "USGS/SRTMGL1_003" # <-- MUDANÇA

# Parâmetros da Exportação/Download
SCALE_METERS = 30 # <-- MUDANÇA (Escala nativa do SRTM)
MAX_PIXELS = 1e13

# Regra para criar a pasta de destino local se não existir
try:
    os.makedirs(OUTUPUT_FOLDER_MDE, exist_ok=True)
    print(f"Pasta local de destino verificada/criada em: {OUTUPUT_FOLDER_MDE}")
except OSError as e:
    print(f"Erro ao criar a pasta local: {e}")
    sys.exit(1)

# =========================================================================
# 1. INICIALIZAÇÃO DO EARTH ENGINE
# =========================================================================
try:
    ee.Initialize()
    print("Google Earth Engine inicializado com sucesso.")
except Exception as e:
    print(f"Erro ao inicializar o GEE. Verifique sua autenticação: {e}")
    sys.exit(1)

# =========================================================================
# 2. CARREGAMENTO E PROCESSAMENTO DOS DADOS
# =========================================================================
print("\nIniciando o processamento de dados...")

try:
    # Carregar a área de interesse (ROI) do arquivo GeoJSON local
    print(f"Carregando ROI do arquivo local: {ROI_LOCAL_PATH}")
    gdf_roi = gpd.read_file(ROI_LOCAL_PATH)

    # Converter a geometria para um ee.FeatureCollection
    roi_geojson = gdf_roi.geometry.__geo_interface__
    ee_features = [ee.Feature(ee.Geometry(feature['geometry'])) for feature in roi_geojson['features']]
    roi_feature_collection = ee.FeatureCollection(ee_features)
    roi_geometry = roi_feature_collection.geometry()
    print("ROI carregada e convertida para geometria do GEE.")

    # --- INÍCIO DAS MUDANÇAS NA SEÇÃO 2 ---

    # Carregar o Dataset SRTM
    print(f"Carregando o dataset SRTM: {SRTM_ID}")
    srtm_image = ee.Image(SRTM_ID)

    # Selecionar a banda de elevação (a banda do SRTM chama-se 'elevation')
    srtm_elevation = srtm_image.select('elevation')
    print("Dataset SRTM (banda 'elevation') carregado.")

    # NOTA: Não é estritamente necessário clipar (srtm_elevation.clip()) aqui,
    # pois a função getDownloadURL usará o parâmetro 'region'
    # para definir a extensão exata do download.

    # --- FIM DAS MUDANÇAS NA SEÇÃO 2 ---

except Exception as e:
    print(f"Erro no processamento dos dados: {e}")
    sys.exit(1)

# =========================================================================
# 3. DOWNLOAD DIRETO PARA PASTA LOCAL
# =========================================================================
print("\nIniciando o download direto do GeoTIFF...")

try:
    # Obter a URL de download do GEE
    url = srtm_elevation.getDownloadURL({ # <-- MUDANÇA (usando srtm_elevation)
        'scale': SCALE_METERS,           # <-- MUDANÇA (usando 30m)
        'crs': 'EPSG:4326',              # WGS84
        'region': roi_geometry.bounds().getInfo()['coordinates'], # <-- MANTIDO (usando a mesma ROI)
        'format': 'GeoTIFF'
    })
    print("URL de download gerada pelo GEE.")

    # Definir o nome e o caminho completo do arquivo de saída
    output_filename = f"{EXPORT_DESCRIPTION.lower().replace(' ', '_')}.tif" # <-- MUDANÇA (baseado na nova descrição)
    local_filepath = os.path.join(OUTUPUT_FOLDER_MDE, output_filename) # <-- MANTIDO (mesma pasta)

    # Fazer o download do arquivo
    print(f"Baixando arquivo para: {local_filepath}...")
    urllib.request.urlretrieve(url, local_filepath)

    print(f"\n🎉 DOWNLOAD CONCLUÍDO com sucesso!")
    print(f"Arquivo salvo em: {local_filepath}")

except Exception as e:
    print(f"\n❌ FALHA NO DOWNLOAD! Mensagem de Erro: {e}")
    sys.exit(1)

print("\nProcesso finalizado.")

## DOWNLOAD AD_SOLOS

Como sabemos a base Água disponível no solo (EMBRAPA) é uma base vetorial. Desta forma rasterizei a base para aproximadamento 30m e subi como uma imagem utilizando a estrutura do google earth engine. Esse dataset esta disponibilizado publicamente pelo meu GEE pelo link :

In [1]:
import ee
import time
import sys
import os
import geopandas as gpd
import urllib.request

# =========================================================================
# 0. DEFINIÇÕES DE CAMINHO LOCAL E CONFIGURAÇÃO
# =========================================================================

# Variáveis Locais
ROI_LOCAL_PATH = r'D:\POS\!FW2_HAND_osmnx_4s_op3n\2_organizacao_fo_fluxo\valida\inundacao.geojson' # ⬅️ CAMINHO
# <-- MUDANÇA: Atualizando a pasta de destino
OUTUPUT_FOLDER_AD_SOLO = r'ad_solo'
# <-- MUDANÇA: Nome do arquivo final exato, conforme solicitado
OUTPUT_FILENAME = 'AD_solos.tif'


# Variáveis do Google Earth Engine
# <-- MUDANÇA: ID do seu asset de solos
ASSET_ID = "projects/ee-phccasagrande20/assets/AD_Solos_30m"

# Parâmetros da Exportação/Download
# SCALE_METERS será detectado automaticamente na Seção 2
MAX_PIXELS = 1e13

# Regra para criar a pasta de destino local se não existir
try:
    os.makedirs(OUTUPUT_FOLDER_AD_SOLO, exist_ok=True) # <-- MUDANÇA: Usando a nova pasta
    print(f"Pasta local de destino verificada/criada em: {OUTUPUT_FOLDER_AD_SOLO}")
except OSError as e:
    print(f"Erro ao criar a pasta local: {e}")
    sys.exit(1)

# =========================================================================
# 1. INICIALIZAÇÃO DO EARTH ENGINE
# =========================================================================
try:
    ee.Initialize()
    print("Google Earth Engine inicializado com sucesso.")
except Exception as e:
    print(f"Erro ao inicializar o GEE. Verifique sua autenticação: {e}")
    sys.exit(1)

# =========================================================================
# 2. CARREGAMENTO E PROCESSAMENTO DOS DADOS
# =========================================================================
print("\nIniciando o processamento de dados...")

try:
    # Carregar a área de interesse (ROI) do arquivo GeoJSON local
    print(f"Carregando ROI do arquivo local: {ROI_LOCAL_PATH}")
    gdf_roi = gpd.read_file(ROI_LOCAL_PATH)

    # Converter a geometria para um ee.FeatureCollection
    roi_geojson = gdf_roi.geometry.__geo_interface__
    ee_features = [ee.Feature(ee.Geometry(feature['geometry'])) for feature in roi_geojson['features']]
    roi_feature_collection = ee.FeatureCollection(ee_features)
    roi_geometry = roi_feature_collection.geometry()
    print("ROI carregada e convertida para geometria do GEE.")

    # --- INÍCIO DAS MUDANÇAS NA SEÇÃO 2 ---

    # Carregar o Dataset de Solos (seu asset)
    print(f"Carregando o asset: {ASSET_ID}")
    solos_image = ee.Image(ASSET_ID) # <-- MUDANÇA

    # Detectar a escala nativa da imagem para usar no download
    print("Obtendo a escala nativa da imagem...")
    native_projection = solos_image.projection()
    SCALE_METERS = native_projection.nominalScale().getInfo() # <-- MUDANÇA
    print(f"Escala nativa detectada: {SCALE_METERS} metros")

    # --- FIM DAS MUDANÇAS NA SEÇÃO 2 ---

except Exception as e:
    print(f"Erro no processamento dos dados: {e}")
    sys.exit(1)

# =========================================================================
# 3. DOWNLOAD DIRETO PARA PASTA LOCAL
# =========================================================================
print("\nIniciando o download direto do GeoTIFF...")

try:
    # Obter a URL de download do GEE
    url = solos_image.getDownloadURL({ # <-- MUDANÇA (usando solos_image)
        'scale': SCALE_METERS,           # <-- MUDANÇA (usando escala nativa)
        'crs': 'EPSG:4326',              # WGS84
        'region': roi_geometry.bounds().getInfo()['coordinates'], # <-- MANTIDO (mesma ROI)
        'format': 'GeoTIFF'
    })
    print("URL de download gerada pelo GEE.")

    # Definir o nome e o caminho completo do arquivo de saída
    output_filename = OUTPUT_FILENAME # <-- MUDANÇA (usando o nome exato)
    local_filepath = os.path.join(OUTUPUT_FOLDER_AD_SOLO, output_filename) # <-- MUDANÇA (usando a pasta nova)

    # Fazer o download do arquivo
    print(f"Baixando arquivo para: {local_filepath}...")
    urllib.request.urlretrieve(url, local_filepath)

    print(f"\n🎉 DOWNLOAD CONCLUÍDO com sucesso!")
    print(f"Arquivo salvo em: {local_filepath}")

except Exception as e:
    print(f"\n❌ FALHA NO DOWNLOAD! Mensagem de Erro: {e}")
    sys.exit(1)

print("\nProcesso finalizado.")

C:\Users\G15-PHCCG\miniconda3\envs\hydro_mde\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Pasta local de destino verificada/criada em: ad_solo
Google Earth Engine inicializado com sucesso.

Iniciando o processamento de dados...
Carregando ROI do arquivo local: D:\POS\!FW2_HAND_osmnx_4s_op3n\2_organizacao_fo_fluxo\valida\inundacao.geojson
ROI carregada e convertida para geometria do GEE.
Carregando o asset: projects/ee-phccasagrande20/assets/AD_Solos_30m
Obtendo a escala nativa da imagem...
Escala nativa detectada: 30.056313971649583 metros

Iniciando o download direto do GeoTIFF...
URL de download gerada pelo GEE.
Baixando arquivo para: ad_solo\AD_solos.tif...

🎉 DOWNLOAD CONCLUÍDO com sucesso!
Arquivo salvo em: ad_solo\AD_solos.tif

Processo finalizado.


## DOWNLOAD OSMNX